<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M6_MLOps_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial, we'll build an Iris classification app using three layers: Data Layer (SQLite), Business Layer (Machine Learning training), and Presentation Layer (HTML, CSS). We'll use Python and its Flask web framework to connect everything together.

Prerequisites:

- Python installed
- Flask library installed
- SQLite
- Basic knowledge of HTML, CSS

![](https://raw.githubusercontent.com/aaubs/ds-master/main/data/Images/SSI-AWS.jpg)

####1. Data Layer: Setting up SQLite
First, let's create a SQLite database for the Iris dataset. SQLite is a lightweight, easy-to-use, serverless SQL database engine.

Create a new file named **database.py** and paste the following code:


In [6]:
#database.py
import sqlite3
import pandas as pd

def init_db():
  # Load the Iris dataset into a Pandas DataFrame
  url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
  df = pd.read_csv(url, header=None, names=["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])

  # Connect to the SQLite database
  conn = sqlite3.connect("iris.db")

  # Save the Pandas DataFrame to the SQLite database
  df.to_sql("iris", conn, if_exists="replace", index=False)

  # Close the connection to the SQLite database
  conn.close()

if __name__ == '__main__':
    init_db()


Now, run database.py to create the database and the iris_data table.


In [7]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("iris.db")

# Read the list of tables using Pandas
tables_df = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

# Print the table names
for table_name in tables_df['name']:
    print(table_name)

# Close the connection
conn.close()

iris


####2. Business Layer: Training the Machine Learning Model
We'll use the scikit-learn library to train a machine learning model for Iris classification. Install scikit-learn if you haven't already:
Create a new file named **ml_model.py** and paste the following code:



In [8]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

def train_model():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)

    with open("model.pkl", "wb") as f:
        pickle.dump(clf, f)

    return clf.score(X_test, y_test)

if __name__ == '__main__':
    accuracy = train_model()
    print(f"Model trained with accuracy: {accuracy}")


Model trained with accuracy: 1.0


Run ml_model.py to train the model and save it as model.pkl.



####3. Presentation Layer: HTML & CSS
Create a new folder named **templates**, and inside it, create a new file named **index.html**. Paste the following code:

![](https://raw.githubusercontent.com/aaubs/ds-master/main/data/Images/html_css_java.jpg)

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Iris Classification</title>
    <style>
        /* Add your CSS styling here */
    </style>
</head>
<body>
    <h1>Iris Classification</h1>
    <form action="/classify" method="post">
        <label for="sepal_length">Sepal Length:</label>
        <input type="number" step="0.1" id="sepal_length" name="sepal_length" required><br><br>
        <label for="sepal_width">Sepal Width:</label>
        <input type="number" step="0.1" id="sepal_width" name="sepal_width" required><br><br>
        <label for="petal_length">Petal Length:</label>
        <input type="number" step="0.1" id="petal_length" name="petal_length" required><br><br>
        <label for="petal_width">Petal Width:</label>
        <input type="number" step="0.1" id="petal_width" name="petal_width" required><br><br>
        <input type="submit" value="Classify">
    </form>
    {% if prediction %}
    <h2>Prediction: {{ prediction }}</h2>
    {% endif %}
</body>
</html>



Now, let's create a basic CSS file to style our app. Create a new folder named **static**, and inside it, create a new file named **style.css**. Paste the following code:

In [ ]:
body {
    font-family: Arial, sans-serif;
    max-width: 600px;
    margin: 0 auto;
    padding: 20px;
}

input[type=number], input[type=submit] {
    width: 100%;
    padding: 5px;
    margin: 5px 0;
    box-sizing: border-box;
}

input[type=submit] {
    background-color: #4CAF50;
    color: white;
    cursor: pointer;
}


####4. Connecting Everything with Flask
Create a new file named **app.py** and paste the following code:



In [ ]:
from flask import Flask, render_template, request, jsonify
import pickle
import sqlite3

app = Flask(__name__)

with open("model.pkl", "rb") as f:
    model = pickle.load(f)

@app.route("/", methods=["GET"])
def index():
    return render_template("index.html", prediction=None)

@app.route("/classify", methods=["POST"])
def classify():
    sepal_length = float(request.form["sepal_length"])
    sepal_width = float(request.form["sepal_width"])
    petal_length = float(request.form["petal_length"])
    petal_width = float(request.form["petal_width"])

    data = [[sepal_length, sepal_width, petal_length, petal_width]]
    prediction = model.predict(data)[0]
    species = ["Setosa", "Versicolor", "Virginica"]

    # Save the data to the database
    connection = sqlite3.connect("iris.db")
    cursor = connection.cursor()
    cursor.execute("INSERT INTO iris (sepal_length, sepal_width, petal_length, petal_width, species) VALUES (?, ?, ?, ?, ?)",
                   (sepal_length, sepal_width, petal_length, petal_width, species[prediction]))
    connection.commit()
    connection.close()

    return jsonify({"prediction": species[prediction]})

if __name__ == "__main__":
    app.run(debug=True, port=5002)


Run app.py and navigate to http://127.0.0.1:5000/ in your web browser. You should see the Iris Classification app, where you can input the features and receive a classification prediction. The data will also be saved to the SQLite database.